In [1]:
import pandas as pd
import numpy as np
import torch
from scipy.spatial.distance import cdist
import os # Để tạo thư mục logs

# --- Phần 1: Cài đặt thư viện và Imports ---
print(">>> Bước 1: Kiểm tra và cài đặt thư viện (nếu cần)...")
try:
    from tsl.data import ImputationDataset, SpatioTemporalDataModule
    from tsl.data.preprocessing import StandardScaler
    from tsl.data.datamodule.splitters import TemporalSplitter
    from tsl.nn.models import GRINModel
    from tsl.metrics import numpy as numpy_metrics
    from tsl.metrics import torch as torch_metrics
    from tsl.engines import Imputer
    from tsl.utils.casting import torch_to_numpy
    print("Thư viện tsl đã được cài đặt.")
except ImportError:
    print("Đang cài đặt thư viện tsl và các phụ thuộc...")
    # Lưu ý: Lệnh pip có thể cần điều chỉnh tùy thuộc môi trường (Colab, Kaggle, local)
    # và phiên bản CUDA của bạn. Đây là ví dụ cho PyTorch 2.x và CUDA 12.1.
    # Thay đổi '+cu121' nếu bạn dùng phiên bản CUDA khác hoặc CPU.
    !pip install torch-scatter -f https://data.pyg.org/whl/torch-2.5.1+cu121.html
    !pip install torch-sparse -f https://data.pyg.org/whl/torch-2.5.1+cu121.html
    !pip install torch-geometric
    !pip install torch-spatiotemporal
    print("Cài đặt hoàn tất.")
    # Import lại sau khi cài đặt
    from tsl.data import ImputationDataset, SpatioTemporalDataModule
    from tsl.data.preprocessing import StandardScaler
    from tsl.data.datamodule.splitters import TemporalSplitter
    from tsl.nn.models import GRINModel
    from tsl.metrics import numpy as numpy_metrics
    from tsl.metrics import torch as torch_metrics
    from tsl.engines import Imputer
    from tsl.utils.casting import torch_to_numpy

from torch.utils.data import DataLoader # Import DataLoader chuẩn
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

>>> Bước 1: Kiểm tra và cài đặt thư viện (nếu cần)...
Đang cài đặt thư viện tsl và các phụ thuộc...
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 49.3 MB/s eta 0:00:0000:0100:01
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 21.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 6.4 MB/s eta 0:00:00a 0:00:01
Cài đặt hoàn tất.


In [2]:
atmospheric_path = "/kaggle/input/satellite-based-dataset/atmospheric_factors_df.csv"
atmospheric_factors_df = pd.read_csv(atmospheric_path)
satellite_based_factors = ['CLOUD', 'CO', 'HCHO', 'NO2', 'O3', 'SO2', 'AAI']
atmospheric_factors_df['date'] = pd.to_datetime(atmospheric_factors_df['date'])

In [3]:
def convert_pivot_df(df: pd.DataFrame, factors=satellite_based_factors) -> pd.DataFrame:
    
    converted_df = df.melt(
        id_vars=['date', 'ID'],
        value_vars=factors,
        var_name="satellite_based_factors",
        value_name="value"
    )
    
    pivot_df = converted_df.pivot_table(
        index='date',
        columns=['ID', "satellite_based_factors"],
        values="value",
        fill_value=np.nan,
        dropna=False
    )
    
    return pivot_df

def calculate_adj_matrix(df: pd.DataFrame) -> tuple:
    station_ids = df['ID'].unique()
    coord_list = []

    for id in station_ids:
        id_coord = list(df.loc[df['ID'] == id][['lon', 'lat']].iloc[0])
        coord_list.append(id_coord)

    # Tính khoảng cách giữa các trạm
    distances = cdist(coord_list, coord_list, metric="euclidean")

    # Thêm epsilon nhỏ để tránh chia cho 0
    epsilon = 1e-10

    # Trích xuất edge index
    edge_index = np.array(np.nonzero(distances))

    # Tính edge weight dựa trên nghịch đảo khoảng cách
    edge_weight = 1.0 / (distances[edge_index[0], edge_index[1]] + epsilon)

    # Chuẩn hóa theo từng node một cách an toàn
    for i in range(len(station_ids)):
        mask = (edge_index[0] == i)
        if mask.sum() > 0:
            weight_sum = edge_weight[mask].sum()
            if weight_sum > epsilon:  # Tránh chia cho giá trị gần bằng 0
                edge_weight[mask] = edge_weight[mask] / weight_sum

    return edge_index, edge_weight.astype(np.float32)

def calculate_mask_matrix(pivot_df: pd.DataFrame) -> pd.DataFrame:
    return (~pivot_df.isna()).astype(bool)

def calculate_covariates(pivot_df: pd.DataFrame) -> pd.DataFrame:
    # Sử dụng cả ngày và tháng làm đặc trưng phụ
    day_features = pivot_df.index.day.values.reshape(-1, 1)
    month_features = pivot_df.index.month.values.reshape(-1, 1)
    return np.concatenate([day_features, month_features], axis=1)

def create_balanced_eval_mask(pivot_df, ratio=0.4):
    """Tạo mask đánh giá cân bằng, chỉ sử dụng một phần dữ liệu không thiếu để đánh giá."""

    # Mask dữ liệu có sẵn
    available_mask = ~pivot_df.isna()

    # Tạo mask ngẫu nhiên với tỷ lệ ratio% dữ liệu có sẵn
    np.random.seed(42)  # Đảm bảo tính tái lập
    random_mask = np.random.rand(*available_mask.shape) < ratio

    # Mask đánh giá chỉ bao gồm dữ liệu có sẵn
    eval_mask = available_mask & random_mask

    return eval_mask

In [4]:
# Tạo pivot_df, adj và các mặt nạ
pivot_df = convert_pivot_df(df=atmospheric_factors_df)
adj = calculate_adj_matrix(df=atmospheric_factors_df)
covariates = calculate_covariates(pivot_df)
mask_matrix = calculate_mask_matrix(pivot_df)
eval_mask = create_balanced_eval_mask(pivot_df, ratio=0.2)

# Kiểm tra thông tin mask
print(f"Số lượng giá trị có sẵn: {mask_matrix.sum().sum()}")
print(f"Số lượng giá trị dùng cho đánh giá: {eval_mask.sum().sum()}")
print(f"Tỷ lệ đánh giá/có sẵn: {eval_mask.sum().sum() / mask_matrix.sum().sum():.2f}")

Số lượng giá trị có sẵn: 248005
Số lượng giá trị dùng cho đánh giá: 49545
Tỷ lệ đánh giá/có sẵn: 0.20


In [5]:
# Kiểm tra các cột có phương sai bằng 0
variances = pivot_df.var()
zero_variance_cols = variances[variances == 0].index
if not zero_variance_cols.empty:
    print(f"Cảnh báo: Có cột có phương sai bằng 0: {zero_variance_cols}")
    print("Thêm nhiễu nhỏ để tăng phương sai...")

    # Thêm nhiễu nhỏ vào cột có phương sai bằng 0
    for col in zero_variance_cols:
        mask = ~pivot_df[col].isna()
        if mask.sum() > 0:
            # Thêm nhiễu nhỏ vào giá trị không phải NaN
            mean_val = pivot_df.loc[mask, col].mean()
            pivot_df.loc[mask, col] += np.random.normal(0, 0.001, size=mask.sum())

# Kiểm tra giá trị outlier
for col in pivot_df.columns:
    values = pivot_df[col].dropna()
    if len(values) > 0:
        q1, q3 = values.quantile([0.25, 0.75])
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # Đếm outlier
        outliers = values[(values < lower_bound) | (values > upper_bound)]
        if len(outliers) > 0:
            print(f"Cột {col}: {len(outliers)} outlier được phát hiện")

Cột (5, 'AAI'): 58 outlier được phát hiện
Cột (5, 'CO'): 41 outlier được phát hiện
Cột (5, 'HCHO'): 18 outlier được phát hiện
Cột (5, 'NO2'): 33 outlier được phát hiện
Cột (5, 'SO2'): 22 outlier được phát hiện
Cột (19, 'AAI'): 48 outlier được phát hiện
Cột (19, 'CO'): 32 outlier được phát hiện
Cột (19, 'HCHO'): 19 outlier được phát hiện
Cột (19, 'NO2'): 47 outlier được phát hiện
Cột (19, 'O3'): 1 outlier được phát hiện
Cột (19, 'SO2'): 24 outlier được phát hiện
Cột (51, 'AAI'): 55 outlier được phát hiện
Cột (51, 'CO'): 48 outlier được phát hiện
Cột (51, 'HCHO'): 18 outlier được phát hiện
Cột (51, 'NO2'): 32 outlier được phát hiện
Cột (51, 'SO2'): 28 outlier được phát hiện
Cột (52, 'AAI'): 61 outlier được phát hiện
Cột (52, 'CO'): 35 outlier được phát hiện
Cột (52, 'HCHO'): 19 outlier được phát hiện
Cột (52, 'NO2'): 63 outlier được phát hiện
Cột (52, 'SO2'): 24 outlier được phát hiện
Cột (53, 'AAI'): 62 outlier được phát hiện
Cột (53, 'CO'): 43 outlier được phát hiện
Cột (53, 'HCHO'): 2

In [6]:
# Tạo dataset dùng cho imputation
imputation_dataset = ImputationDataset(
    target=pivot_df,
    eval_mask=eval_mask,  # Sử dụng mask được cân bằng
    covariates={'time': covariates},  # Thêm đặc trưng thời gian
    connectivity=adj,  # Sử dụng ma trận kề cải tiến
    window=14,  # Giảm kích thước cửa sổ xuống
    stride=1
)

In [7]:
# Cấu hình datamodule với scale dữ liệu robust
scalers = {'target': StandardScaler(axis=(0, 1))}

splitter = TemporalSplitter(
    val_len=0.1,
    test_len=0.2
)

dm = SpatioTemporalDataModule(
    dataset=imputation_dataset,
    scalers=scalers,
    splitter=splitter,
    batch_size=64,  # Giảm kích thước batch để ổn định hơn
    workers=4
)
dm.setup()

In [8]:
# Kiểm tra eval_mask trên batch từ val_dataloader
val_dataloader = dm.val_dataloader()
for batch in val_dataloader:
    val_eval_mask = batch['eval_mask']
    num_samples = val_eval_mask.shape[0]  # Số mẫu trong batch
    num_true = val_eval_mask.sum().item()
    print(f"Batch có {num_samples} mẫu, số lượng giá trị True trong val_eval_mask: {num_true}")

    # Kiểm tra nếu có quá ít giá trị True
    if num_true < 10:
        print("CẢNH BÁO: Quá ít điểm đánh giá trong batch!")
    # break  # Chỉ kiểm tra batch đầu tiên

Batch có 64 mẫu, số lượng giá trị True trong val_eval_mask: 22198
Batch có 64 mẫu, số lượng giá trị True trong val_eval_mask: 21797
Batch có 32 mẫu, số lượng giá trị True trong val_eval_mask: 11228


In [9]:
model_cls = GRINModel
model_kwargs = dict(
    n_nodes=imputation_dataset.n_nodes,
    input_size=imputation_dataset.n_channels,
    hidden_size=64,        # Tăng kích thước hidden để bắt xu hướng phức tạp
    ff_size=128,           # Tăng feedforward size
    embedding_size=8,      # Tăng kích thước embedding
    n_layers=2,            # Thêm một lớp để tăng khả năng học
    kernel_size=2,
    decoder_order=1,
    layer_norm=True,
    dropout=0.1,           # Điều chỉnh dropout phù hợp
    ff_dropout=0.1,
    merge_mode='mlp'
)

loss_fn = torch_metrics.MaskedMAE()

log_metrics = {
    'mae': torch_metrics.MaskedMAE(),
    'mse': torch_metrics.MaskedMSE(),
    'mre': torch_metrics.MaskedMRE(),
    'mape': torch_metrics.MaskedMAPE()
}

scheduler_class = torch.optim.lr_scheduler.CosineAnnealingLR
epochs = 50  # Giảm số epoch
scheduler_kwargs = dict(
    eta_min=0.0001,
    T_max=epochs
)

In [10]:
imputer = Imputer(
    model_class=model_cls,
    model_kwargs=model_kwargs,
    optim_class=torch.optim.Adam,
    optim_kwargs={
        "lr": 0.0005,          # Giảm learning rate
        "weight_decay": 1e-4    # Tăng weight decay
    },
    loss_fn=loss_fn,
    metrics=log_metrics,
    scheduler_class=scheduler_class,
    scheduler_kwargs=scheduler_kwargs,
    scale_target=True,
    whiten_prob=0.1,          # Tăng whiten_prob
    prediction_loss_weight=1.0,
    impute_only_missing=True,
    warm_up_steps=10           # Thêm warm up steps
)

In [11]:
logger = TensorBoardLogger(save_dir="logs", name="imputation")
early_stop_callback = EarlyStopping(monitor="val_mae", patience=15, mode="min")
checkpoint_callback = ModelCheckpoint(
    dirpath="logs/imputation",  # Sửa thành logs thay vì log
    save_top_k=1,
    monitor="val_mae",
    mode="min"
)

callbacks = [early_stop_callback, checkpoint_callback]

In [12]:
trainer = Trainer(
    max_epochs=epochs,
    logger=logger,
    callbacks=callbacks,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    gradient_clip_val=1.0,  # Giảm grad clip để tránh exploding gradients
    enable_progress_bar=True,
    detect_anomaly=True,    # Bật phát hiện anomaly để debug
    check_val_every_n_epoch=1
)

trainer.fit(imputer, datamodule=dm)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /kaggle/working/logs/imputation exists and is not empty.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (24) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [13]:
imputer.load_model(checkpoint_callback.best_model_path)
imputer.freeze()
trainer.test(imputer, datamodule=dm)

/usr/local/lib/python3.10/dist-packages/tsl/engines/predictor.py:121: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  storage = torch.load(filename, lambda storage, loc: stora

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │            0.0            │
│         test_mae          │    0.06523766368627548    │
│         test_mape         │     14539.8544921875      │
│         test_mre          │     0.737855076789856     │
│         test_mse          │    0.02220136485993862    │
└───────────────────────────┴───────────────────────────┘

[{'test_mae': 0.06523766368627548,
  'test_mape': 14539.8544921875,
  'test_mre': 0.737855076789856,
  'test_mse': 0.02220136485993862,
  'test_loss': 0.0}]

In [14]:
# Tạo empty eval mask cho toàn bộ dataset
def create_empty_eval_mask(pivot_df):
    """Tạo mask đánh giá rỗng (toàn False) có cùng định dạng với dữ liệu gốc."""
    eval_mask = pivot_df.isna().astype(int)
    eval_mask = (eval_mask * 0).astype(bool)
    return eval_mask

# Tạo dataset mới để impute toàn bộ dữ liệu
imputation_dataset_full = ImputationDataset(
    target=pivot_df,  # Toàn bộ dữ liệu
    eval_mask=create_empty_eval_mask(pivot_df),  # Không cần eval_mask
    covariates={'time': covariates},  # Sử dụng covariates đã tạo trước đó
    connectivity=adj,  # Sử dụng ma trận kề đã tạo trước đó
    window=14,
    stride=1
)

# Cấu hình datamodule cho toàn bộ dữ liệu
dm_full = SpatioTemporalDataModule(
    dataset=imputation_dataset_full,
    scalers=scalers,  # Sử dụng cùng scaler đã dùng trong huấn luyện
    splitter=None,  # Không cần chia dữ liệu
    batch_size=64,
    workers=4
)
dm_full.setup()

# Tạo DataLoader cho toàn bộ dữ liệu
dm_full.trainset = list(range(len(imputation_dataset_full)))
full_dataloader = dm_full.train_dataloader(shuffle=False)

In [15]:
# Tạo hàm wrapper cho tsl.data.batch.StaticBatch để xử lý lỗi tensor shape khi predict
from tsl.data.batch import StaticBatch

def create_compatible_batch(batch_dict):
    """Chuyển đổi dictionary batch thành đối tượng StaticBatch."""
    batch = StaticBatch()
    
    # Tạo thuộc tính input
    batch.input = type('InputContainer', (), {})()
    
    # Thêm các tensor cơ bản
    batch.input.x = batch_dict['x']
    batch.input.mask = batch_dict['mask']
    
    # Thêm các tensor tùy chọn
    if 'edge_index' in batch_dict:
        batch.input.edge_index = batch_dict['edge_index']
    if 'edge_weight' in batch_dict:
        batch.input.edge_weight = batch_dict['edge_weight']
    if 'u' in batch_dict:
        batch.input.u = batch_dict['u']
    
    # Thêm các thuộc tính khác
    if 'y' in batch_dict:
        batch.y = batch_dict['y']
    batch.mask = batch_dict['mask']
    if 'eval_mask' in batch_dict:
        batch.eval_mask = batch_dict['eval_mask']
    if 'transform' in batch_dict:
        batch.transform = batch_dict['transform']
    else:
        batch.transform = {}
    
    # Thiết lập kích thước batch
    batch.batch_size = batch_dict['x'].size(0)
    
    return batch

In [16]:
# Hàm dự đoán an toàn để xử lý các vấn đề kích thước tensor
def safe_predict(imputer_model, dataloader, device):
    """Dự đoán an toàn trên từng mẫu một."""
    imputer_model.eval()  # Đặt mô hình về chế độ đánh giá
    results = []
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            # Chuyển đổi sang StaticBatch tương thích
            compatible_batch = create_compatible_batch(batch)
            
            try:
                # Sử dụng predict_step để xử lý toàn bộ batch
                output = imputer_model.predict_step(compatible_batch, batch_idx)
                results.append(output)
                
                # In tiến trình
                if (batch_idx + 1) % 5 == 0:
                    print(f"Đã xử lý {batch_idx + 1}/{len(dataloader)} batch")
            except Exception as e:
                print(f"Lỗi khi xử lý batch {batch_idx}: {e}")
                # Xử lý từng mẫu nếu xử lý cả batch gặp lỗi
                batch_results = []
                
                # Chuyển dữ liệu sang device
                x = batch['x'].to(device)
                mask = batch['mask'].to(device)
                edge_index = batch.get('edge_index', None)
                if edge_index is not None:
                    edge_index = edge_index.to(device)
                edge_weight = batch.get('edge_weight', None)
                if edge_weight is not None:
                    edge_weight = edge_weight.to(device)
                u = batch.get('u', None)
                if u is not None:
                    u = u.to(device)
                    
                # Xử lý từng mẫu
                for i in range(x.size(0)):
                    sample_x = x[i:i+1]  # Giữ batch dimension
                    sample_mask = mask[i:i+1]
                    sample_u = u[i:i+1] if u is not None else None
                    
                    # Chuẩn bị đầu vào
                    input_dict = {
                        'x': sample_x,
                        'mask': sample_mask
                    }
                    if edge_index is not None:
                        input_dict['edge_index'] = edge_index
                    if edge_weight is not None:
                        input_dict['edge_weight'] = edge_weight
                    if sample_u is not None:
                        input_dict['u'] = sample_u
                        
                    try:
                        # Gọi forward và lấy kết quả
                        output = imputer_model.model.forward(**input_dict)
                        if isinstance(output, list) and len(output) > 0:
                            imputation = output[0]
                        else:
                            imputation = output
                            
                        # Xử lý inverse transform
                        trans = batch.get('transform', {}).get('y')
                        if trans is not None:
                            imputation = trans.inverse_transform(imputation)
                            
                        batch_results.append(imputation)
                    except Exception as e:
                        print(f"Lỗi khi dự đoán mẫu {i} trong batch {batch_idx}: {e}")
                        
                # Kết hợp kết quả của batch
                if batch_results:
                    batch_imputation = torch.cat(batch_results, dim=0)
                    
                    batch_output = {
                        'y': batch.get('y', torch.zeros_like(batch_imputation)).cpu(),
                        'y_hat': batch_imputation.cpu(),
                        'mask': batch['mask'].cpu(),
                        'eval_mask': batch.get('eval_mask', torch.zeros_like(batch['mask'])).cpu()
                    }
                    
                    results.append(batch_output)
                    
    return results

In [17]:
# Thực hiện dự đoán trên toàn bộ dữ liệu
print("Thực hiện imputation cho những điểm missing thực sự trong dữ liệu...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Phương pháp 1: Sử dụng trainer.predict (đơn giản nhất nhưng có thể gặp lỗi tensor shape)
try:
    print("\nPhương pháp 1: Sử dụng trainer.predict()...")
    predictions1 = trainer.predict(imputer, dataloaders=full_dataloader)
    print(f"Đã dự đoán thành công với {len(predictions)} batch")
except Exception as e:
    print(f"Lỗi khi sử dụng trainer.predict(): {e}")
    predictions = None

# Phương pháp 2: Sử dụng trainer.test (thường hoạt động tốt hơn)
try:
    print("\nPhương pháp 2: Sử dụng trainer.test()...")
    test_results = trainer.test(imputer, datamodule=dm_full)
    print(f"Test metrics: {test_results}")
except Exception as e:
    print(f"Lỗi khi sử dụng trainer.test(): {e}")

# Phương pháp 3: Sử dụng hàm safe_predict (đáng tin cậy nhất)
print("\nPhương pháp 3: Sử dụng safe_predict()...")
predictions = safe_predict(imputer, full_dataloader, device)
print(f"Đã dự đoán thành công với {len(predictions)} batch")

Thực hiện imputation cho những điểm missing thực sự trong dữ liệu...

Phương pháp 1: Sử dụng trainer.predict()...


Predicting: |          | 0/? [00:00<?, ?it/s]

Lỗi khi sử dụng trainer.predict(): The size of tensor a (64) must match the size of tensor b (14) at non-singleton dimension 1

Phương pháp 2: Sử dụng trainer.test()...
Lỗi khi sử dụng trainer.test(): An invalid dataloader was returned from `SpatioTemporalDataModule.test_dataloader()`. Found None.

Phương pháp 3: Sử dụng safe_predict()...
Đã xử lý 5/34 batch
Đã xử lý 10/34 batch
Đã xử lý 15/34 batch
Đã xử lý 20/34 batch
Đã xử lý 25/34 batch
Đã xử lý 30/34 batch
Đã dự đoán thành công với 34 batch


In [18]:
method1_output = imputer.collate_prediction_outputs(predictions)
method1_output = torch_to_numpy(method1_output)

In [19]:
# Xử lý kết quả dự đoán để tạo DataFrame imputed
print("Xử lý kết quả dự đoán...")

if True:
    # Hàm để kết hợp kết quả từ các batch
    def collate_prediction_outputs(outputs):
        print("Hello world")
        processed_res = dict()
        keys = set()
        # Lặp qua kết quả từng batch
        for res in outputs:
            for k, v in res.items():
                if k in keys:
                    processed_res[k].append(v)
                else:
                    processed_res[k] = [v]
                keys.add(k)
        # Nối kết quả
        for k, v in processed_res.items():
            if all(torch.is_tensor(x) for x in v):
                processed_res[k] = torch.cat(v, 0)
        return processed_res
    
    # Kết hợp kết quả từ các batch
    output = collate_prediction_outputs(predictions)
    output = torch_to_numpy(output)
    
    print("Thông tin về kết quả dự đoán:")
    print(f"- Kích thước y_hat: {output['y_hat'].shape}")
    
    # Nếu có dữ liệu thực tế, tính metrics
    if 'y' in output and output['y'] is not None:
        if 'mask' in output and output['mask'] is not None:
            mask = output['mask']
            print(f"- Số điểm có dữ liệu (mask=True): {mask.sum()}")
            
            # Tính metrics trên dữ liệu đã biết
            print("\nMetrics trên dữ liệu đã biết:")
            print(f"- MAE: {numpy_metrics.mae(output['y_hat'], output['y'], mask)}")
            print(f"- MSE: {numpy_metrics.mse(output['y_hat'], output['y'], mask)}")
            print(f"- RMSE: {numpy_metrics.rmse(output['y_hat'], output['y'], mask)}")
else:
    print("Không có kết quả dự đoán!")

Xử lý kết quả dự đoán...
Hello world
Thông tin về kết quả dự đoán:
- Kích thước y_hat: (2176, 14, 26, 7)


In [20]:
# Cải tiến hàm điền giá trị vào DataFrame
print("Đang tạo DataFrame với giá trị đã được imputed (phiên bản cải tiến)...")

if 'output' in locals() and 'y_hat' in output:
    # Tạo bản sao của DataFrame gốc
    imputed_df = pivot_df.copy()
    
    # Xác định vị trí các giá trị bị thiếu
    missing_mask = imputed_df.isna()
    num_missing = missing_mask.sum().sum()
    print(f"Tổng số giá trị bị thiếu: {num_missing}")
    
    try:
        # Phân tích kích thước dự đoán và cấu trúc DataFrame
        predictions_shape = output['y_hat'].shape
        print(f"Kích thước tensor dự đoán: {predictions_shape}")
        print(f"Số hàng trong DataFrame: {len(imputed_df)}")
        print(f"Số cột trong DataFrame: {len(imputed_df.columns)}")
        
        # Kiểm tra cấu trúc index của các cột
        print(f"Cấu trúc index của cột: {type(imputed_df.columns)}")
        
        # Xử lý kết quả dự đoán để khớp với định dạng của DataFrame
        if len(predictions_shape) == 4:  # [samples, window, nodes, features]
            print("Xử lý tensor 4D...")
            # Lấy giá trị ở middle time step
            middle_step = predictions_shape[1] // 2
            predictions = output['y_hat'][:, middle_step, :, :]
            print(f"Kích thước sau khi lấy middle time step: {predictions.shape}")
        elif len(predictions_shape) == 3:  # [window, nodes, features] hoặc [samples, nodes, features]
            print("Xử lý tensor 3D...")
            if predictions_shape[0] > len(imputed_df):
                # Trường hợp này là [samples, nodes, features]
                predictions = output['y_hat']
            else:
                # Trường hợp này là [window, nodes, features]
                middle_step = predictions_shape[0] // 2
                predictions = output['y_hat'][middle_step, :, :]
            print(f"Kích thước sau khi xử lý: {predictions.shape}")
        else:
            predictions = output['y_hat']
            print(f"Sử dụng tensor với kích thước: {predictions.shape}")
        
        # Tạo mapping từ node và feature index sang DataFrame columns
        column_map = {}
        node_ids = []
        feature_ids = []
        
        # Xác định tất cả các node và feature từ MultiIndex
        try:
            # Trường hợp MultiIndex (ID, feature)
            if isinstance(imputed_df.columns, pd.MultiIndex):
                print("Xử lý MultiIndex columns...")
                for i, (node, feature) in enumerate(imputed_df.columns):
                    if node not in node_ids:
                        node_ids.append(node)
                    if feature not in feature_ids:
                        feature_ids.append(feature)
                    column_map[(node_ids.index(node), feature_ids.index(feature))] = i
            else:
                # Trường hợp Index đơn giản
                print("Xử lý Index đơn giản...")
                for i, col in enumerate(imputed_df.columns):
                    column_map[i] = i
        except Exception as e:
            print(f"Lỗi khi phân tích cấu trúc cột: {e}")
            # Tạo mapping đơn giản dựa trên chỉ số
            for i in range(len(imputed_df.columns)):
                column_map[i] = i
        
        print(f"Số node phát hiện: {len(node_ids)}")
        print(f"Số feature phát hiện: {len(feature_ids)}")
        
        # Điền giá trị thiếu trong DataFrame
        print("Đang điền giá trị vào các vị trí bị thiếu...")
        
        # Đếm số lượng giá trị đã được điền
        filled_count = 0
        skipped_count = 0
        
        # Phương pháp 1: Điền trực tiếp dùng mapping
        if isinstance(imputed_df.columns, pd.MultiIndex):
            # Duyệt qua từng ngày trong dữ liệu
            for date_idx, date in enumerate(imputed_df.index):
                if date_idx < predictions.shape[0]:
                    # Duyệt qua từng node và feature
                    for node_idx, node_id in enumerate(node_ids):
                        if node_idx < predictions.shape[1]:
                            for feat_idx, feat in enumerate(feature_ids):
                                if feat_idx < predictions.shape[2]:
                                    try:
                                        # Xây dựng cột MultiIndex
                                        col = (node_id, feat)
                                        if col in imputed_df.columns and missing_mask.loc[date, col]:
                                            # Điền giá trị dự đoán
                                            imputed_df.loc[date, col] = predictions[date_idx, node_idx, feat_idx]
                                            filled_count += 1
                                    except Exception as e:
                                        print(f"Lỗi khi điền giá trị cho ngày {date}, node {node_id}, feature {feat}: {e}")
                                        skipped_count += 1
        else:
            # Phương pháp 2: Điền trực tiếp theo chỉ số đơn giản
            dates = imputed_df.index.tolist()
            for date_idx, date in enumerate(dates):
                if date_idx < predictions.shape[0]:
                    for col_idx, col in enumerate(imputed_df.columns):
                        if missing_mask.loc[date, col]:
                            node_idx = col_idx // imputation_dataset.n_channels
                            feat_idx = col_idx % imputation_dataset.n_channels
                            
                            if node_idx < predictions.shape[1] and feat_idx < predictions.shape[2]:
                                try:
                                    # Điền giá trị dự đoán
                                    imputed_df.loc[date, col] = predictions[date_idx, node_idx, feat_idx]
                                    filled_count += 1
                                except Exception as e:
                                    print(f"Lỗi khi điền giá trị đơn giản cho {date}, cột {col}: {e}")
                                    skipped_count += 1
        
        print(f"Đã điền {filled_count}/{num_missing} giá trị bị thiếu.")
        if skipped_count > 0:
            print(f"Đã bỏ qua {skipped_count} giá trị do lỗi.")
        
        # Kiểm tra lại số lượng missing sau khi impute
        remaining_missing = imputed_df.isna().sum().sum()
        print(f"Số lượng giá trị còn thiếu sau khi impute: {remaining_missing}")
        missing_pct = remaining_missing / imputed_df.size * 100
        print(f"Phần trăm giá trị còn thiếu: {missing_pct:.2f}%")
        
        # Nếu vẫn còn nhiều giá trị thiếu, thử phương pháp khác
        if remaining_missing > 0.3 * num_missing:
            print("\nVẫn còn nhiều giá trị thiếu, thử phương pháp khác...")
            
            # Phương pháp 3: Thử điền giá trị bằng cách reshape tensor
            try:
                # Reshape predictions để phù hợp với DataFrame
                flat_predictions = predictions.reshape(predictions.shape[0], -1)
                print(f"Kích thước tensor sau khi reshape: {flat_predictions.shape}")
                
                # Kiểm tra từng hàng và thử điền giá trị thiếu
                for date_idx, date in enumerate(imputed_df.index):
                    if date_idx < flat_predictions.shape[0]:
                        for col_idx, col in enumerate(imputed_df.columns):
                            if missing_mask.loc[date, col] and col_idx < flat_predictions.shape[1]:
                                imputed_df.loc[date, col] = flat_predictions[date_idx, col_idx]
                
                # Kiểm tra lại sau khi thử phương pháp mới
                remaining_missing_new = imputed_df.isna().sum().sum()
                improved = num_missing - remaining_missing_new
                print(f"Phương pháp mới đã điền thêm được {improved} giá trị.")
                print(f"Số lượng giá trị còn thiếu: {remaining_missing_new}")
            except Exception as e:
                print(f"Lỗi khi thử phương pháp mới: {e}")
        
        # Lưu DataFrame đã được imputed
        imputed_df.to_csv('atmospheric_imputed.csv')
        print("Đã lưu DataFrame đã được imputed vào file 'atmospheric_imputed.csv'")
        
        # Hiển thị một số ví dụ về giá trị đã được điền
        print("\nMột số ví dụ về giá trị đã được điền:")
        sample_cols = list(imputed_df.columns[:3])
        sample_rows = imputed_df.index[:5]
        print(imputed_df.loc[sample_rows, sample_cols])
        
        # Kiểm tra tỷ lệ missing của từng node
        print("\nTỷ lệ giá trị còn thiếu theo từng node:")
        try:
            if isinstance(imputed_df.columns, pd.MultiIndex):
                node_missing = {}
                for node in node_ids:
                    node_cols = [col for col in imputed_df.columns if col[0] == node]
                    missing_count = imputed_df[node_cols].isna().sum().sum()
                    total_count = len(node_cols) * len(imputed_df)
                    node_missing[node] = (missing_count, missing_count/total_count*100)
                
                for node, (count, pct) in node_missing.items():
                    print(f"Node {node}: {count} giá trị thiếu ({pct:.2f}%)")
        except Exception as e:
            print(f"Lỗi khi phân tích missing theo node: {e}")
    except Exception as e:
        print(f"Lỗi khi điền giá trị vào DataFrame: {e}")
else:
    print("Không có dữ liệu dự đoán để tạo DataFrame imputed.")

Đang tạo DataFrame với giá trị đã được imputed (phiên bản cải tiến)...
Tổng số giá trị bị thiếu: 150939
Kích thước tensor dự đoán: (2176, 14, 26, 7)
Số hàng trong DataFrame: 2192
Số cột trong DataFrame: 182
Cấu trúc index của cột: <class 'pandas.core.indexes.multi.MultiIndex'>
Xử lý tensor 4D...
Kích thước sau khi lấy middle time step: (2176, 26, 7)
Xử lý MultiIndex columns...
Số node phát hiện: 26
Số feature phát hiện: 7
Đang điền giá trị vào các vị trí bị thiếu...
Đã điền 150033/150939 giá trị bị thiếu.
Số lượng giá trị còn thiếu sau khi impute: 906
Phần trăm giá trị còn thiếu: 0.23%
Đã lưu DataFrame đã được imputed vào file 'atmospheric_imputed.csv'

Một số ví dụ về giá trị đã được điền:
ID                              5                    
satellite_based_factors       AAI     CLOUD        CO
date                                                 
2019-01-01              -1.112399  1.000000  0.047875
2019-01-02              -0.346883  1.000000  0.043722
2019-01-03              -0.590